In [1]:
import json
import engine
import numpy as np
from num2words import num2words
from IPython.display import display
from PIL import Image
from pprint import pprint as print
from tqdm import tqdm

In [2]:
generator = engine.QGenerator()
data_prefix = 'val'
scenes = json.load(open(f'{data_prefix}_sceneGraphs.json', 'r'))
img_ids = set(json.load(open('valid_img_ids.json')))
scenes = {k: scenes[k] for k in list(scenes.keys()) for k in img_ids}

In [3]:
assert set(scenes.keys()) == img_ids

In [4]:
question_family = 'color_query'
question = json.load(open(f'question_families/{question_family}.json', 'r'))
program = question['program']
text_templates = question['templates']

In [5]:
question = json.load(open(f'question_families/{question_family}.json', 'r'))

In [6]:
print(question)

{'constraints': ['unique(obj1)', 'exclude_color(attrs1)'],
 'name': 'color_query',
 'program': ['color', ['scene', 'obj1', 'attrs1']],
 'templates': ['What color is the <attrs1> <obj1>?'],
 'tokens': ['obj1', 'attrs1', 'color1']}


In [7]:
print(question['templates'])

['What color is the <attrs1> <obj1>?']


In [8]:
vocab = set(json.load(open('val_label_vocab.json')))

In [9]:
question

{'name': 'color_query',
 'constraints': ['unique(obj1)', 'exclude_color(attrs1)'],
 'program': ['color', ['scene', 'obj1', 'attrs1']],
 'templates': ['What color is the <attrs1> <obj1>?'],
 'tokens': ['obj1', 'attrs1', 'color1']}

In [10]:
%%capture
# with open(f'{data_prefix}_{question_family}.txt', 'w+') as f:
d_names = ['question_id', 'img_id', 'label', 'sent']
bool_binary = {True: 'yes', False: 'no'}
all_questions = []
for img_id, scene in tqdm(scenes.items()):
#     qa_pairs = list(map(lambda x: dict(zip(d_names, (img_id, *x))), qa_pairs))
    for ix, (question_type, sent, answer) in enumerate(generator.generate_questions(scene, question)):
        q_id = str(img_id) + str(ix).zfill(9)
        answer_dict = {answer: 1.0}
        question_dict = {'img_id': img_id,
                         'label': answer_dict,
                         'question_id': q_id,
                         'sent': sent,
                         'question_type': question_type}
        all_questions.append(question_dict)

In [11]:
question_family = 'color_query_bool'
question = json.load(open(f'question_families/{question_family}.json', 'r'))
program = question['program']
text_templates = question['templates']

In [12]:
%%capture
bool_binary = {True: 'yes', False: 'no'}
for img_id, scene in tqdm(scenes.items()):
#     qa_pairs = list(map(lambda x: dict(zip(d_names, (img_id, *x))), qa_pairs))
    for ix, (question_type, sent, answer) in enumerate(generator.generate_questions(scene, question)):
        q_id = str(img_id) + str(ix).zfill(9)
        answer_dict = {bool_binary[answer]: 1.0}
        question_dict = {'img_id': img_id,
                         'label': answer_dict,
                         'question_id': q_id,
                         'sent': sent,
                         'question_type': question_type}
        all_questions.append(question_dict)

In [16]:
json.dump(all_questions, open(f'{question_family}.json', 'w+'))

In [110]:
all_questions

[{'img_id': '2408684',
  'label': {'brown': 1.0},
  'question_id': '2408684000000000',
  'sent': 'What color is the striped lying cat?',
  'question_type': 'color_query-0'},
 {'img_id': '2408684',
  'label': {'brown': 1.0},
  'question_id': '2408684000000001',
  'sent': 'What color is the resting cat?',
  'question_type': 'color_query-0'},
 {'img_id': '2408684',
  'label': {'brown': 1.0},
  'question_id': '2408684000000002',
  'sent': 'Is the furry upside down cat brown?',
  'question_type': 'color_query-1'},
 {'img_id': '2408684',
  'label': {'white': 1.0},
  'question_id': '2408684000000003',
  'sent': 'What color is the curtain?',
  'question_type': 'color_query-0'},
 {'img_id': '2408684',
  'label': {'brown': 1.0},
  'question_id': '2408684000000004',
  'sent': 'Is the resting striped cat brown?',
  'question_type': 'color_query-1'},
 {'img_id': '2408684',
  'label': {'brown': 1.0},
  'question_id': '2408684000000005',
  'sent': 'Is the resting furry upside down lying cat brown?',


In [8]:
import os

In [43]:
all_ans = set()
train_root = '../vqa_diag/questions1.2/val_all_questions.json'
# for file in os.listdir(train_root):
filename = f'{train_root}'
all_ans = all_ans.union({a['answer'] for a in json.load(open(filename)).values()})
print(f'{filename}: {len(all_ans)}')
# all_qs = json.load(open('../vqa_diag/questions1.2/train_all_questions/train_all_questions_0.json'))

'../vqa_diag/questions1.2/val_all_questions.json: 1501'


In [67]:
all_qs = json.load(open(filename))

In [68]:
all_qs

{'05451384': {'semantic': [{'operation': 'select',
    'dependencies': [],
    'argument': 'cat (1324563)'},
   {'operation': 'filter color', 'dependencies': [0], 'argument': 'brown'},
   {'operation': 'relate',
    'dependencies': [1],
    'argument': 'blanket,under,s (-)'},
   {'operation': 'exist', 'dependencies': [2], 'argument': '?'}],
  'entailed': ['05451386', '05451385'],
  'equivalent': ['05451384'],
  'question': 'Are there blankets under the brown cat?',
  'imageId': '2382986',
  'isBalanced': False,
  'groups': {'global': None, 'local': '13-cat_blanket'},
  'answer': 'no',
  'semanticStr': 'select: cat (1324563)->filter color: brown [0]->relate: blanket,under,s (-) [1]->exist: ? [2]',
  'annotations': {'answer': {},
   'question': {'5:7': '1324563'},
   'fullAnswer': {'4': '1324576', '7': '1324563'}},
  'types': {'detailed': 'existRelSC',
   'semantic': 'rel',
   'structural': 'verify'},
  'fullAnswer': 'No, there is a towel under the cat.'},
 '05451385': {'semantic': [{'op

In [10]:
filename

'../vqa_diag/questions1.2/train_all_questions/train_all_questions_1.json'